In [ ]:
!pip install gdown -q
print("Downloading folder from Drive...")
# Downloads the folder structure containing the Linemod dataset
!gdown "https://drive.google.com/file/d/1Zwh-gAk_-CBgpOcNLPLdFNxggi3NTh-S/view?usp=drive_link" --fuzzy
import glob
zip_files = glob.glob("**/Linemod_preprocessed.zip", recursive=True)

if zip_files:
    zip_path = zip_files[0]
    print(f"Unzipping {zip_path}...")
    !unzip -q -o "{zip_path}"
    print("Extraction complete!")
else:
    print("Error: Linemod_preprocessed.zip not found. Check the download.")

Cloning into '6D_pose'...
remote: Enumerating objects: 174, done.
remote: Counting objects: 100% (174/174), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 174 (delta 87), reused 113 (delta 45), pack-reused 0 (from 0)
Receiving objects: 100% (174/174), 2.23 MiB | 6.95 MiB/s, done.
Resolving deltas: 100% (87/87), done.
Cloned https://github.com/fraco03/6D_pose.git to /content/6D_pose


In [ ]:
import os
import sys

# Clone or pull part
repo_url = "https://github.com/fraco03/6D_pose.git"
repo_dir = "/kaggle/working/6D_pose"   #Modify here for kaggle
branch = "main"

# Clone if missing
if not os.path.exists(repo_dir):
    !git clone -b {branch} {repo_url}
    print(f"Cloned {repo_url} to {repo_dir}")
else:
    %cd {repo_dir}
    !git fetch origin
    !git checkout {branch}
    !git reset --hard origin/{branch}
    %cd ..
    print(f"Updated {repo_url} to {repo_dir}")

# Add repository to Python path
if repo_dir not in sys.path:
    sys.path.insert(0, repo_dir)

In [ ]:
# Cancella tutte le cartelle __pycache__ ricorsivamente nella directory di lavoro
!find . -name "__pycache__" -type d -exec rm -rf {} +
print("🗑️ Cache pulita dal disco.")

In [ ]:
!pip install plyfile
from src.pose_rgb.dataset import LineModPoseDataset
from src.pose_rgb.model import ResNetRotation, TranslationNet
from src.pose_rgb.pose_utils import quaternion_to_rotation_matrix, convert_rotation_to_quaternion, inverse_pinhole_projection
from src.pose_rgb.test_dataset import *
from src.pose_rgb.loss import CombinedPoseLoss, MultiObjectPointMatchingLoss, TranslationLoss
from torch.utils.data import Dataset, DataLoader
import pathlib
import torch.optim as optim
from tqdm import tqdm
from utils.projection_utils import *
from utils.linemod_config import *
from metrics import compute_ADD_metric_quaternion


In [ ]:
root_dir = '/kaggle/input/line-mode/Linemod_preprocessed' #Modify here for kaggle

train_dataset = LineModPoseDataset(split='train', root_dir=root_dir)
test_dataset = LineModPoseDataset(split='test', root_dir=root_dir)

#Dataloder
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

 Loaded LineModPoseDataset
   Split: train
   Dir : [1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15]
   Total samples: 3631
 Loaded LineModPoseDataset
   Split: test
   Dir : [1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15]
   Total samples: 20528


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
!pip install trimesh
import torch
import numpy as np
import trimesh
import os

def load_all_object_points(models_dir, valid_obj_ids, num_points=1000):
    """
    Loads .ply files for ALL objects and stacks them into a single Tensor.
    
    Args:
        models_dir (str): Folder containing .ply files (e.g., 'obj_01.ply').
        valid_obj_ids (list): List of integers IDs (e.g., [1, 5, 6...]).
        num_points (int): Number of points to sample per object.
        
    Returns:
        torch.Tensor: Shape (Num_Classes, num_points, 3).
                      The index in dimension 0 corresponds to the index in valid_obj_ids.
    """
    all_points_list = []
    
    print(f"📦 Loading {len(valid_obj_ids)} 3D models from {models_dir}...")
    
    for i, obj_id in enumerate(valid_obj_ids):
        # Construct filename assuming LineMod format (e.g., 'obj_01.ply')
        ply_name = f"obj_{obj_id:02d}.ply" 
        ply_path = os.path.join(models_dir, ply_name)
        
        if not os.path.exists(ply_path):
            raise FileNotFoundError(f"Model not found: {ply_path}")

        # Load mesh
        mesh = trimesh.load(ply_path)
        vertices = np.array(mesh.vertices)
        
        # Sample points
        if len(vertices) > num_points:
            idx = np.random.choice(len(vertices), num_points, replace=False)
            selected = vertices[idx]
        else:
            # Padding via repetition if not enough points (rare in LineMod)
            choice = np.random.choice(len(vertices), num_points, replace=True)
            selected = vertices[choice]
            
        # Add to list
        all_points_list.append(selected)

    # Stack into a single tensor
    # Shape: (Num_Classes, Num_Points, 3)
    # Example: (13, 1000, 3)
    bank_tensor = torch.from_numpy(np.array(all_points_list)).float()
    
    # Unit conversion (mm to meters) if needed
    # bank_tensor = bank_tensor / 1000.0 
    
    return bank_tensor / 1000

In [ ]:
LINEMOD_NAMES = [
            'ape',         # Index 0 (ID 1)
            'benchvise',   # Index 1 (ID 2)
            'camera',      # Index 2 (ID 4)
            'can',         # Index 3 (ID 5)
            'cat',         # Index 4 (ID 6)
            'driller',     # Index 5 (ID 8)
            'duck',        # Index 6 (ID 9)
            'eggbox',      # Index 7 (ID 10)
            'glue',        # Index 8 (ID 11)
            'holepuncher', # Index 9 (ID 12)
            'iron',        # Index 10 (ID 13)
            'lamp',        # Index 11 (ID 14)
            'phone'        # Index 12 (ID 15)
        ]
name_to_idx = {name: i for i, name in enumerate(LINEMOD_NAMES)}

# ROTATION ONLY

In [ ]:
# ONLY ROTATION TRAINING SCRIPT
import os
import torch
import torch.optim as optim
from tqdm import tqdm
import matplotlib.pyplot as plt
import json
from datetime import datetime
from itertools import islice
import numpy as np

# ==========================================
# 1. SETUP & HYPERPARAMETERS
# ==========================================
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LEARNING_RATE = 0.0001
NUM_EPOCHS = 50

# --- PATHS ---
# Define where your .ply models are located
MODELS_DIR = '/kaggle/input/line-mode/Linemod_preprocessed/models' 
# List of valid object IDs in your dataset (must match your dataset logic)
VALID_OBJ_IDS = [1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15] 

# --- LOGGING SETUP ---
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
# Directory to save checkpoints and logs
CHECKPOINT_DIR = f'/kaggle/working/run_rotation' 
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
run_dir = CHECKPOINT_DIR

print(f"\n🔥 STARTING ROTATION-ONLY TRAINING on {DEVICE}...")
print(f"📁 Saving outputs to: {run_dir}")

# ==========================================
# 2. INITIALIZE LOSS & MODELS
# ==========================================

# A. LOAD 3D POINTS FOR LOSS
# We need to load the point clouds for all objects to use PointMatchingLoss.
print("📦 Loading 3D Point Clouds for Loss Function...")
# Use the helper function we defined earlier to load all ply files
point_bank = load_all_object_points(MODELS_DIR, VALID_OBJ_IDS, num_points=1000)
point_bank = point_bank.to(DEVICE) # Move entire bank to GPU


# B. DEFINE LOSS FUNCTION

criterion = MultiObjectPointMatchingLoss(point_bank).to(DEVICE)

# C. INITIALIZE MODEL
# We only use the Rotation Network
model_rot = ResNetRotation(freeze_backbone=False).to(DEVICE)

# D. OPTIMIZER
# We only optimize the rotation model parameters
optimizer = optim.Adam(
    model_rot.parameters(),
    lr=LEARNING_RATE
)

# E. METRICS STORAGE
train_losses = []
val_losses = []
best_val_loss = float('inf')

# ==========================================
# 3. TRAINING LOOP
# ==========================================
for epoch in range(NUM_EPOCHS):

    # --- A. TRAIN PHASE ---
    model_rot.train()
    running_train_loss = 0.0

    # Progress Bar
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]")

    for batch in pbar:
        # 1. Move data to GPU
        imgs = batch['image'].to(DEVICE)
        gt_rot = batch['rotation'].to(DEVICE)
        
        # We need class indices for the PointMatchingLoss (Index 0 to 12)
        # Ensure your Dataset returns 'class_id' as a mapped index (0..N), NOT the raw Linemod ID (1,5,8..)
        raw_names_list = batch['class_idx'] # es. ['can', 'ape', 'driller']
        
        
        try:
            indices = [name_to_idx[name] for name in raw_names_list]
        except KeyError as e:
            print(f"❌ ERRORE CRITICO: Trovato nome '{e}' non presente nella lista LINEMOD_NAMES!")
            raise e

        
        class_ids = torch.tensor(indices, dtype=torch.long).to(DEVICE)

        # 2. Forward Pass
        pred_rot = model_rot(imgs)

        # 3. Calculate Loss
        # Pass class_ids so the loss knows which 3D model to use for each image in the batch
        if point_bank is not None:
            loss = criterion(pred_rot, gt_rot, class_ids)
        else:
            loss = criterion(pred_rot, gt_rot) # Fallback doesn't use IDs

        # 4. Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 5. Logging
        running_train_loss += loss.item()
        pbar.set_postfix({'ADD Loss': f"{loss.item():.4f}"})

    avg_train_loss = running_train_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # --- B. EVALUATION PHASE ---
    model_rot.eval()
    running_val_loss = 0.0
    val_batches_limit = 50  # Validate on a subset to save time per epoch
    count_batches = 0

    with torch.no_grad():
        val_iterator = islice(test_loader, val_batches_limit)
        val_pbar = tqdm(val_iterator, total=val_batches_limit, desc="Validating")

        for batch in val_pbar:
            imgs = batch['image'].to(DEVICE)
            gt_rot = batch['rotation'].to(DEVICE)
            raw_names_list = batch['class_idx'] # es. ['can', 'ape', 'driller']
            
            
            try:
                indices = [name_to_idx[name] for name in raw_names_list]
            except KeyError as e:
                print(f"❌ ERRORE CRITICO: Trovato nome '{e}' non presente nella lista LINEMOD_NAMES!")
                raise e
    
            
            class_ids = torch.tensor(indices, dtype=torch.long).to(DEVICE)

            # Forward
            pred_rot = model_rot(imgs)

            # Loss
            if point_bank is not None:
                loss = criterion(pred_rot, gt_rot, class_ids)
            else:
                loss = criterion(pred_rot, gt_rot)
                
            running_val_loss += loss.item()
            count_batches += 1

    avg_val_loss = running_val_loss / count_batches if count_batches > 0 else 0
    val_losses.append(avg_val_loss)

    # --- C. REPORT & SAVE ---
    print(f"📊 Epoch {epoch+1} Summary: Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

    # Save Best Model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        save_path = os.path.join(CHECKPOINT_DIR, "best_model_rot.pth")
        torch.save({
            'epoch': epoch,
            'model_state_dict': model_rot.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': best_val_loss
        }, save_path)
        print(f"🏆 New Best Rotation Model Saved! (Loss: {best_val_loss:.4f})")

    # Save Last Checkpoint (for resuming if needed)
    if (epoch + 1) == NUM_EPOCHS:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model_rot.state_dict(),
            'val_loss': avg_val_loss
        }, os.path.join(CHECKPOINT_DIR, f"checkpoint_last.pth"))

# --- D. PLOTTING ---
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Train Loss (ADD Metric)')
plt.plot(val_losses, label='Val Loss (ADD Metric)')
plt.title('Rotation Training Convergence')
plt.xlabel('Epochs')
plt.ylabel('Average Distance (m)')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(CHECKPOINT_DIR, 'rotation_training_curve.png'))
print("🎉 TRAINING COMPLETE! Training curve saved.")

# Translation ONLY

In [ ]:
# ONLY TRANSLATION TRAINING SCRIPT
from src.pose_rgb import model
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import os
import numpy as np

# ==========================================
# 1. CONFIGURATION
# ==========================================
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 64        # Adjust if you run out of memory
LR = 0.001             # Constant Learning Rate
NUM_EPOCHS = 60
CHECKPOINT_DIR = f'/kaggle/working/run_translation' 
DATA_ROOT = '/kaggle/input/line-mode/Linemod_preprocessed'

os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# ==========================================
# 2. MODEL & OPTIMIZER SETUP
# ==========================================
print(f"🧠 Initializing Model on {DEVICE}...")

# Initialize your custom TranslationNet
model_transl = TranslationNet().to(DEVICE)

# Define Loss (Weighted to prioritize Depth Z)
criterion = TranslationLoss(z_weight=1) 

# Simple Adam Optimizer (No Scheduler)
optimizer = optim.Adam(model_transl.parameters(), lr=LR)

# ==========================================
# 3. TRAINING LOOP
# ==========================================
best_val_mae = float('inf') # Track the best error to save the best model

print("🚀 Starting Training Loop...")

for epoch in range(NUM_EPOCHS):
    
    # --- TRAIN PHASE ---
    model_transl.train()
    running_loss = 0.0
    
    # Progress bar for training
    train_loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]")
    
    for batch in train_loop:
        # Move data to GPU
        imgs = batch['image'].to(DEVICE)           # (B, 3, 224, 224)
        bbox_info = batch['bbox_info'].to(DEVICE)  # (B, 4) Normalized BBox GPS
        gt_trans = batch['translation'].to(DEVICE) # (B, 3) Absolute Translation in METERS

        # Forward Pass
        preds = model_transl(imgs, bbox_info)
        
        # Calculate Loss
        loss = criterion(preds, gt_trans)
        
        # Backward Pass (Update Weights)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Update stats
        running_loss += loss.item()
        train_loop.set_postfix(loss=loss.item())

    avg_train_loss = running_loss / len(train_loader)

    # --- VALIDATION PHASE ---
    model_transl.eval()
    val_loss = 0.0
    
    # Variables to calculate error in Centimeters (for human readability)
    error_sum_xyz = np.array([0.0, 0.0, 0.0]) 
    total_samples = 0
    
    with torch.no_grad():
        for batch in tqdm(test_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Eval]"):
            imgs = batch['image'].to(DEVICE)
            bbox_info = batch['bbox_info'].to(DEVICE)
            gt_trans = batch['translation'].to(DEVICE)

            # Predict
            preds = model_transl(imgs, bbox_info)
            
            # Calculate Loss
            loss = criterion(preds, gt_trans)
            val_loss += loss.item()
            
            # Calculate Absolute Error (in Meters)
            abs_err = torch.abs(preds - gt_trans).cpu().numpy()
            error_sum_xyz += abs_err.sum(axis=0)
            total_samples += imgs.shape[0]

    avg_val_loss = val_loss / len(test_loader)
    
    # Convert Mean Error to Centimeters
    mean_error_m = error_sum_xyz / total_samples
    mean_error_cm = mean_error_m * 100.0
    total_mae_cm = np.mean(mean_error_cm) # Average error across X, Y, Z

    # --- REPORTING ---
    print(f"\n📊 REPORT EPOCH {epoch+1}")
    print(f"   Train Loss:    {avg_train_loss:.5f}")
    print(f"   Val Loss:      {avg_val_loss:.5f}")
    print(f"   --------------------------------")
    print(f"   Error X:       {mean_error_cm[0]:.2f} cm")
    print(f"   Error Y:       {mean_error_cm[1]:.2f} cm")
    print(f"   Error Z:       {mean_error_cm[2]:.2f} cm (Depth)")
    print(f"   --------------------------------")
    
    # Save Best Model (if error is lower than previous best)
    if total_mae_cm < best_val_mae:
        best_val_mae = total_mae_cm
        torch.save(model_transl.state_dict(), f"{CHECKPOINT_DIR}/best_translation_model.pth")
        print(f"   💾 New Best Model Saved! (Avg Error: {total_mae_cm:.2f} cm)")
        
    # Save periodic checkpoint every 10 epochs
    if (epoch + 1) % 10 == 0:
        torch.save(model_transl.state_dict(), f"{CHECKPOINT_DIR}/translation_ep{epoch+1}.pth")

print("\n✅ Training Complete. Best model saved in:", CHECKPOINT_DIR)

# Evaluation

In [1]:
import sys
sys.path.append('../..')

In [3]:
from src.pose_rgb.evaluate import evaluate_RGB, evaluate_RGB_rot_only

model_path = "./RGB_run/best_model_rot.pth"
dataset_root = "../../Linemod_preprocessed"
output_path = "./RGB_run/evaluation_results.csv"


df = evaluate_RGB_rot_only(
    model_path=model_path,
    dataset_root=dataset_root,
    output_path=output_path
)
    
df

📥 Loading 3D model points and diameters...
📦 Loading trained model...
📚 Preparing test dataset and dataloader...
 Loaded LineModPoseDepthDataset
   Split: test (Ratio: 0.20)
   Objects: [1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15]
   Total samples: 4867

🚀 Starting Comprehensive Benchmark (ADD Error + ADD-0.1d Accuracy)...


Evaluating: 100%|██████████| 39/39 [16:03<00:00, 24.70s/batch]


📊 Evaluation report saved to ./RGB_run/evaluation_results.csv


,Object ID,Object Name,Diameter (mm),Mean ADD (mm),Mean ADD-S (mm),Mean ADD-Rot (mm),Mean ADD-S-Rot (mm),ADD-0.1d Accuracy (%),ADD-S-0.1d Accuracy (%)
0,1,ape,102.098657,6.371118,2.129254,6.371118,2.129254,86.864407,99.788136
1,2,benchvise,247.506242,12.728971,5.129825,12.728971,5.129825,92.148760,100.000000
2,5,can,201.403586,10.640517,3.693451,10.640517,3.693451,93.920335,99.790356
3,6,cat,154.545518,7.686827,3.226671,7.686827,3.226671,92.374728,99.128540
4,8,driller,261.471781,11.043530,5.103864,11.043530,5.103864,96.210526,100.000000
5,9,duck,108.999201,8.140133,2.754954,8.140133,2.754954,83.609959,99.792531
6,10,eggbox,164.627588,10.934605,3.253344,10.934605,3.253344,87.731481,100.000000
7,11,glue,175.889334,8.390186,3.777869,8.390186,3.777869,95.161290,100.000000
8,12,holepuncher,145.542875,9.141986,2.895303,9.141986,2.895303,85.918367,100.000000
9,4,camera,172.492249,8.409860,3.107158,8.409860,3.107158,95.020747,100.000000


In [3]:
from src.pose_rgb.dataset import LineModPoseDataset
dataset_root = "../../Linemod_preprocessed"
test = LineModPoseDataset(split='test', root_dir=dataset_root)

✅ LineModConfig initialized: F:\Magistrale\Advanced Machine Learning\6D_pose\Linemod_preprocessed
 Loaded LineModPoseDataset
   Split: test (Ratio: 0.20)
   Objects: [1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15]
   Total samples: 3163
